In [1]:
#####################################
### Import the relevant libraries ###
#####################################

from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, roc_auc_score
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import psutil
from pathlib import Path
import os
import time
import seaborn as sns
from sklearn.metrics import confusion_matrix
import tf2onnx

2025-02-23 17:41:08.674956: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740332468.685697   17746 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740332468.688749   17746 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-23 17:41:08.700246: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#Convert to PNG
#RUN ONCE
PATH = Path("dataset") / "GTSRB" / "Training"



def convert_dataset_to_png():
    for path,_,files in os.walk(PATH):
        for file in files:
            if file.endswith(".ppm"):
                file_path = Path(path) / file
                new_path = Path(path) / (file[:-4] + ".png")
                img = cv2.imread(file_path)
                cv2.imwrite(new_path,img)

#convert_dataset_to_png()

In [3]:
#Memory usage
def log_memory():
    #Log memory usage
    process = psutil.Process()
    print(f"{process.memory_info().rss / 1000 / 1000}mb")  # in mb 
    

def log_gpu_memory():
    print(tf.config.experimental.get_memory_info('GPU:0'))

log_gpu_memory()
log_memory()

{'current': 0, 'peak': 0}
889.42592mb


I0000 00:00:1740332471.196732   17746 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5564 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:09:00.0, compute capability: 8.6


***
# **DATA**

In [4]:
LABELS = ["20 km/h"
,"30 km/h"
,"50 km/h"
,"60 km/h"
,"70 km/h"
,"80 km/h"
,"End of speed limit"
,"100 km/h"
,"120 km/h"
,"No passing"
,"No passing for large vehicles"
,"Right of way"
,"Priority road"
,"Yield"
,"Stop"
,"No vehicles"
,"Large vehicles prohibited"
,"No entry"
,"General caution"
,"Dangerous curve to the left"
,"Dangerous curve to the right"
,"Double curve"
,"Bumpy road"
,"Slippery road"
,"Road narrows"
,"Road work"
,"Traffic signals"
,"Pedestrians"
,"Children crossing"
,"Bicycle crossing"
,"Ice warning"
,"Wild animals crossing"
,"End of all speed and passing limits"
,"Turn right ahead"
,"Turn left ahead"
,"Ahead only"
,"Go straight or right"
,"Go straight or left"
,"Keep right"
,"Keep left"
,"Roundabout mandatory"
,"End of no passing"
,"End of no passing by large vehicles"]

In [5]:
import onnx
import onnxruntime as rt
import idx2numpy


In [6]:
path = "models/model.onnx"
model = onnx.load(path)

In [7]:
onnx.checker.check_model(model)

In [8]:
images = idx2numpy.convert_from_file("dataset/images.idx")
labels = idx2numpy.convert_from_file("dataset/labels.idx")
print(images.dtype)

>f8


In [9]:
images = images.astype(np.float32)

In [10]:
providers = ['CPUExecutionProvider']
m = rt.InferenceSession(path, providers=providers)
pred = m.run(None, {"input": images})


In [11]:
m.get_inputs()[0].shape

['unk__62', 32, 32]

In [12]:
for i in range(43):
    print(f"Correct {labels[i]}")
    print(f"Predicted {np.argmax(pred[0][i])}")

Correct 27
Predicted 27
Correct 13
Predicted 13
Correct 4
Predicted 4
Correct 7
Predicted 7
Correct 30
Predicted 30
Correct 39
Predicted 39
Correct 35
Predicted 35
Correct 5
Predicted 5
Correct 18
Predicted 18
Correct 16
Predicted 16
Correct 20
Predicted 20
Correct 8
Predicted 8
Correct 34
Predicted 34
Correct 9
Predicted 9
Correct 10
Predicted 10
Correct 36
Predicted 36
Correct 26
Predicted 26
Correct 19
Predicted 19
Correct 15
Predicted 15
Correct 40
Predicted 40
Correct 6
Predicted 6
Correct 32
Predicted 32
Correct 12
Predicted 12
Correct 42
Predicted 42
Correct 14
Predicted 14
Correct 21
Predicted 21
Correct 2
Predicted 2
Correct 37
Predicted 37
Correct 3
Predicted 3
Correct 17
Predicted 17
Correct 29
Predicted 29
Correct 24
Predicted 24
Correct 41
Predicted 41
Correct 23
Predicted 23
Correct 22
Predicted 22
Correct 33
Predicted 33
Correct 31
Predicted 31
Correct 25
Predicted 25
Correct 1
Predicted 1
Correct 28
Predicted 28
Correct 11
Predicted 11
Correct 38
Predicted 38
Correct 0


In [13]:
path = "examples/mnist-robustness/mnist-classifier.onnx"
model = onnx.load(path)

In [14]:
onnx.checker.check_model(model)

In [15]:
images = idx2numpy.convert_from_file("examples/mnist-robustness/t2-images.idx")
labels = idx2numpy.convert_from_file("examples/mnist-robustness/t2-labels.idx")
print(images.dtype)

>f8


In [16]:
images = images.astype(np.float32)

In [17]:
_images = images.reshape((2,28,28,1))

In [18]:
print(images.shape)
_images.shape

(2, 28, 28)


(2, 28, 28, 1)

In [19]:
# 1 1 28 28
abc = np.zeros((1,1,28,28),dtype=np.float32)
abc[0,0] = images[0]
abc[0,0] = images[1]

In [20]:
providers = ['CPUExecutionProvider']
m = rt.InferenceSession(path, providers=providers)
pred = m.run(None, {"Input3" : abc})


In [22]:
m.get_inputs()[0].shape

[1, 1, 28, 28]